<a href="https://colab.research.google.com/github/Mr-S-Mirzoev/Vein-Detection-and-Removal/blob/main/localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies installation

In [1]:
# Clone Mask_RCNNN

!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 17.18 MiB/s, done.
Resolving deltas: 100% (560/560), done.


In [10]:
# Verify the downloading of the model

!sudo apt-get install tree > /dev/null
!tree . -d

.
├── Mask_RCNN
│   ├── assets
│   ├── images
│   ├── mrcnn
│   └── samples
│       ├── balloon
│       ├── coco
│       ├── nucleus
│       └── shapes
└── sample_data

10 directories


In [18]:
# Install the library

%cd Mask_RCNN/
!ls


/content/Mask_RCNN
assets	LICENSE      mrcnn	requirements.txt  setup.cfg
images	MANIFEST.in  README.md	samples		  setup.py


In [19]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/visualize.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/lib/mrcnn
copying mrcnn/__init__.py -> build/lib/mrcnn
copying mrcnn/config.py -> build/lib/mrcnn
copying mrcnn/utils.py -> build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/visualize.py -> build/bdist.linux-x86_64/egg/m

In [21]:
# Confirm Library installation

!pip show mask-rcnn

Name: mask-rcnn
Version: 2.1
Summary: Mask R-CNN for object detection and instance segmentation
Home-page: https://github.com/matterport/Mask_RCNN
Author: Matterport
Author-email: waleed.abdulla@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages/mask_rcnn-2.1-py3.6.egg
Requires: 
Required-by: 


# Dataset preparation

In [28]:
# Cloning the dataset

!git clone https://github.com/experiencor/kangaroo.git
!tree . -d

fatal: destination path 'kangaroo' already exists and is not an empty directory.
.
├── kangaroo
│   ├── annots
│   └── images
├── Mask_RCNN
│   ├── assets
│   ├── build
│   │   ├── bdist.linux-x86_64
│   │   └── lib
│   │       └── mrcnn
│   ├── dist
│   ├── images
│   ├── mask_rcnn.egg-info
│   ├── mrcnn
│   └── samples
│       ├── balloon
│       ├── coco
│       ├── nucleus
│       └── shapes
└── sample_data

19 directories


The look of the xml annotation is like that:

    <annotation>
      <folder>Kangaroo</folder>
      <filename>00001.jpg</filename>
      <path>...</path>
      <source>
        <database>Unknown</database>
      </source>
      <size>
        <width>450</width>
        <height>319</height>
        <depth>3</depth>
      </size>
      <segmented>0</segmented>
      <object>
        <name>kangaroo</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
          <xmin>233</xmin>
          <ymin>89</ymin>
          <xmax>386</xmax>
          <ymax>262</ymax>
        </bndbox>
      </object>
      <object>
        <name>kangaroo</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
          <xmin>134</xmin>
          <ymin>105</ymin>
          <xmax>341</xmax>
          <ymax>253</ymax>
        </bndbox>
      </object>
    </annotation>

In [33]:
from xml.etree import ElementTree
 
# function to extract bounding boxes from an annotation file
def extract_boxes(filename):
	# load and parse the file
	tree = ElementTree.parse(filename)
	# get the root of the document
	root = tree.getroot()
	# extract each bounding box
	boxes = list()
	for box in root.findall('.//bndbox'):
		xmin = int(box.find('xmin').text)
		ymin = int(box.find('ymin').text)
		xmax = int(box.find('xmax').text)
		ymax = int(box.find('ymax').text)
		coors = [xmin, ymin, xmax, ymax]
		boxes.append(coors)
	# extract image dimensions
	width = int(root.find('.//size/width').text)
	height = int(root.find('.//size/height').text)
	return boxes, width, height
 
# extract details form annotation file
boxes, w, h = extract_boxes('kangaroo/annots/00001.xml')
# summarize extracted details
print("Boxes: {}".format(boxes))
print("Shape of the image: ({}, {})".format(w, h))

Boxes: [[233, 89, 386, 262], [134, 105, 341, 253]]
Shape of the image: (450, 319)
